### Chatbot_QA_Data_Augmentation_and_Evaluation

This notebook contains the following sections:
1. Import the dataset and then use LLM to generate question-answer example based on context from training data/RAG-chain
2. LLM assisted evaluation on Generated Question and Predicted Answer
3. Manual Evaluation / Review
4. Human Correction
5. Add generated qa pairs into dataset
6. Save the new dataset as a jsonl file for next round's evaluation use
7. Test loading the new dataset to make sure it works


### Import the dataset and then use LLM to generate question-answer example based on context from training data/RAG-chain

In [93]:
#Import libraries

from langchain.evaluation.qa import QAGenerateChain
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders import HuggingFaceDatasetLoader

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA

import langchain
from langchain.memory import ConversationBufferMemory

from langchain.evaluation.qa import QAEvalChain
from langchain.evaluation.qa import ContextQAEvalChain

from langchain.schema import Document
import copy

import json
from typing import Iterable

from tqdm import tqdm
import time

import pandas as pd
pd.set_option('display.max_colwidth', None) # Set pandas display options to show all characters

import warnings
# warnings.filterwarnings("ignore", category=UserWarning) # Suppress only UserWarnings
warnings.filterwarnings("ignore") # Suppress all Warnings

In [2]:
# Load Data
dataset_name = "MakTek/Customer_support_faqs_dataset"
page_content_column = "answer"
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
data = loader.load()

In [3]:
# Determine batch size

batch_size = len(data)
# batch_size = 10

In [4]:
# Set model and prompt used in the QA-generation chain

custom_qa_gen_prompt_str = """System: You are the manager of Customer Service Support coming up with example questions from customers.
Given the following document, please generate a question and answer based on that document.

Example Format:
<Begin Document>
...
<End Document>
QUESTION: question here
ANSWER: answer here

Instruction: These questions should be detailed and be based explicitly on information in the document. Begin!

<Begin Document>
{doc}
<End Document>"""

custom_qa_gen_prompt = PromptTemplate(input_variable=["doc"], template=custom_qa_gen_prompt_str)

langchain.evaluation.qa.generate_prompt.PROMPT = custom_qa_gen_prompt

llm_model = "llama3.2"
example_gen_chain = QAGenerateChain.from_llm(OllamaLLM(model=llm_model))

langchain.evaluation.qa.generate_prompt.PROMPT 

PromptTemplate(input_variables=['doc'], input_types={}, partial_variables={}, template='System: You are the manager of Customer Service Support coming up with example questions from customers.\nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nInstruction: These questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>')

In [5]:
# Generated new qa examples

start_time = time.time()

new_examples = []
for item in tqdm([{"doc": t} for t in data[:batch_size]], desc="Generating QA Examples"):
    result = example_gen_chain.apply_and_parse([item])
    new_examples.append(result)

# A trick to flaten the list:
new_examples = sum(new_examples, [])

end_time = time.time()
elapsed_time = (end_time - start_time) / 60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

Generating QA Examples: 100%|███████████████████| 200/200 [13:37<00:00,  4.09s/it]

Elapsed time: 13.62 minutes


In [6]:
#Create dataframe for the generated_qa_examples

generated_qa_examples = []

for i in range(batch_size):
    generated_qa_examples.append({"Dataset Index" : str(i),
                                  "Train-set Question" : data[i].metadata['question'],
                                  "Generated Question" : new_examples[i]['qa_pairs']['query'],
                                  "Train-set Context" : data[i].page_content,
                                  "Generated Answer" : new_examples[i]['qa_pairs']['answer']
                                 })

generated_qa_examples_df = pd.DataFrame(generated_qa_examples)


In [7]:
generated_qa_examples_df.head()

,Dataset Index,Train-set Question,Generated Question,Train-set Context,Generated Answer
0,0,How can I create an account?,What is the step-by-step process for creating an account on this website?,"""To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.""","To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."
1,1,What payment methods do you accept?,What payment methods does an online retailer typically accept for processing online orders?,"""We accept major credit cards, debit cards, and PayPal as payment methods for online orders.""","Major credit cards, debit cards, and PayPal."
2,2,How can I track my order?,What steps should I take to track my order?,"""You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.""","To track your order, log into your account and navigate to the 'Order History' section, where you will find the tracking information for your shipment."
3,3,What is your return policy?,What is the timeframe within which customers can initiate a return under the company's return policy?,"""Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.""",30 days
4,4,Can I cancel my order?,What is the condition under which a customer can cancel their order?,"""You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.""",A customer can cancel their order if it has not been shipped yet.


### LLM assisted evaluation on Generated Question and Predicted Answer

In [8]:
# First let's create a chain.
# Let's use the same kind of chain we used in our customer service chatbot for consistency.
# However, for this evaluation, we don't need to use the memory for simplicity.

embedding_model_chosen = "hkunlp/instructor-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf_embeddings = HuggingFaceEmbeddings(model_name=embedding_model_chosen,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)

persist_directory = 'docs/chroma/'

vectordb = Chroma.from_documents(
    documents=data,
    embedding=hf_embeddings,
    persist_directory=persist_directory
)
retriever=vectordb.as_retriever(search_kwargs={"k": 3}, search_type="mmr")

base_prompt_template = """System: You are a ABC-Company customer service representative.
\n\nInstruction: First, if you know the answer: Answer the customer's question based on following context and chat history. Do not mention we have discussed this topic before in the previous conversation or ask any follow up question. Otherwise, if you do not know the answer: simply answer 'I am not sure about the answer, please contact our human service for assistance. Thank You!'.
\n\nContext: {context}
\n\nChat history: {chat_history}
\n\nQuestion: {question}
\n\nOutput Answer: """
prompt_input_list = ["context", "question", "chat_history"]

BASE_PROMPT = PromptTemplate(
            template=base_prompt_template, input_variables=["context", "question", "chat_history"]
        )

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key='question',
    output_key='answer',
    return_messages=True,
)

qa = ConversationalRetrievalChain.from_llm(
    llm=OllamaLLM(model=llm_model, temperature = 0.01),
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    #return_generated_question=True,
    verbose=False,
    combine_docs_chain_kwargs={'prompt': BASE_PROMPT},
    # condense_question_prompt=condense_question_prompt
)


In [9]:
# Make sure memory is clear

memory.clear()
memory.chat_memory.messages

[]

In [10]:
# Prepare the format for the generated question set

prediction_question_list = []
for i in range(batch_size):
    prediction_question_list.append(new_examples[i]['qa_pairs']['query'])


In [11]:
# Feed the generated question into the llm, generate answer and save the context and predicted answer
start_time = time.time()

prediction_answer_list = []
for question in tqdm(prediction_question_list, desc="Predicting answers for generated questions"):
    memory.clear() # This is necessary because otherwise the memory will becomes too long and response will be too slow
    answer = qa(question)  # Generate an answer

    #append question, answer and source_context
    if answer['source_documents'][0].page_content == answer['source_documents'][1].page_content == answer['source_documents'][2].page_content:
        prediction_answer_list.append({"question": question,
                                        "answer": answer['answer'],
                                        "source_context": answer['source_documents'][0].page_content})        
    elif (answer['source_documents'][0].page_content == answer['source_documents'][1].page_content) or (answer['source_documents'][1].page_content == answer['source_documents'][2].page_content):
        prediction_answer_list.append({"question": question,
                                        "answer": answer['answer'],
                                        "source_context": answer['source_documents'][0].page_content + answer['source_documents'][2].page_content})
    else:
        prediction_answer_list.append({"question": question,
                                        "answer": answer['answer'],
                                        "source_context": answer['source_documents'][0].page_content + answer['source_documents'][1].page_content + answer['source_documents'][2].page_content})
end_time = time.time()
elapsed_time = (end_time - start_time) / 60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

Predicting answers for generated questions: 100%|█| 200/200 [07:36<00:00,  2.28s/i

Elapsed time: 7.62 minutes


In [12]:
# create data frame for prediction_answer_list

prediction_answer_list_df = pd.DataFrame(prediction_answer_list)
prediction_answer_list_df.head()

,question,answer,source_context
0,What is the step-by-step process for creating an account on this website?,"To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.","""To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.""""Yes, you can place an order as a guest without creating an account. However, creating an account offers benefits such as order tracking and easier future purchases."""
1,What payment methods does an online retailer typically accept for processing online orders?,"We accept major credit cards, debit cards, and PayPal as payment methods for online orders.","""We accept major credit cards, debit cards, and PayPal as payment methods for online orders.""""Yes, we take the security of your personal and payment details seriously. We use industry-standard encryption and follow strict security protocols to ensure your information is protected."""
2,What steps should I take to track my order?,"To track your order, please log into your account and navigate to the 'Order History' section. There, you will find the tracking information for your shipment.","""You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.""""If a product is available for pre-order, you can place an order to secure your item. The product will be shipped once it becomes available."""
3,What is the timeframe within which customers can initiate a return under the company's return policy?,"Customers can initiate a return within 30 days of purchase for a full refund, provided they are in their original condition and packaging.","""Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.""""Yes, you can return a product purchased during a promotional event. The refund will be processed based on the amount paid after any applicable discounts."""
4,What is the condition under which a customer can cancel their order?,"You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.","""You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.""""If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.""""If you need to change your order, please contact our customer support team as soon as possible. We will do our best to accommodate your request if the order has not been processed yet."""


In [13]:
# Create a new grading prompt to edit the roles in the evaluation chain

custom_grading_prompt_str = """System: You are a Customer Service Manager grading a Customer Service Chatbot's responses to customers' questions.
You are given a question, the context the question is about, and the Customer Service Chatbot's answer. You are asked to score the Chatbot's answer as either CORRECT or INCORRECT, based on the context.

Instruction: Grade the chatbot answer INCORRECT if it makes up more details than the provided context. Grade the chatbot answer as CORRECT even if the chatbot answer misses some details and does not mention all the context information provided. Ignore differences in punctuation and phrasing between the chatbot answer and context. 

Example Format:
CUSTOMER QUESTION: customer question here
CONTEXT: context the question is about here
CUSTOMER SERVICE CHATBOT ANSWER: customer service chatbot's answer here
GRADE: CORRECT or INCORRECT here

CUSTOMER QUESTION: {query}
CONTEXT: {context}
CUSTOMER SERVICE CHATBOT ANSWER: {result}
GRADE: """

custom_grading_prompt = PromptTemplate(input_variable=["query", "context", "result"], template=custom_grading_prompt_str)


In [14]:
langchain.evaluation.qa.eval_prompt.CONTEXT_PROMPT = custom_grading_prompt
langchain.evaluation.qa.eval_prompt.PROMPT = custom_grading_prompt

In [15]:
# Check out the prompt used in the context evaluation chain
# langchain.evaluation.qa.eval_prompt.CONTEXT_PROMPT

In [16]:
# Check out the prompt used in the context evaluation chain
# langchain.evaluation.qa.eval_prompt.PROMPT

In [17]:
llm=OllamaLLM(model=llm_model)
# eval_chain = QAEvalChain.from_llm(llm)
eval_chain = ContextQAEvalChain.from_llm(llm, prompt = custom_grading_prompt)

In [18]:
# Evaluate the answer

start_time = time.time()
print("...Start Evaluating Answers...")
graded_outputs = eval_chain.evaluate(examples=prediction_answer_list, 
                                     predictions=prediction_answer_list, 
                                     question_key="question", 
                                     context_key="source_context", 
                                     prediction_key="answer")
print("...Done Evaluating Answers...")
end_time = time.time()
elapsed_time = (end_time - start_time) / 60
print(f"Elapsed time: {elapsed_time:.2f} minutes")

...Start Evaluating Answers...
...Done Evaluating Answers...
Elapsed time: 19.10 minutes


In [45]:
total_incorrect_counter = 0
total_correct_counter = 0
human_incorrect_ans_review = []
human_correct_ans_review = []

for i, eg in enumerate(prediction_answer_list):
    
    if "INCORRECT" in graded_outputs[i]['text']:
        total_incorrect_counter += 1
        append_list = human_incorrect_ans_review
        
    elif "CORRECT" in graded_outputs[i]['text']:
        total_correct_counter += 1
        append_list = human_correct_ans_review

    append_list.append({"Generate-Index" : i,
                             "Generated Question" :prediction_answer_list[i]['question'],
                              "Context Answer" :prediction_answer_list[i]['source_context'],
                              "Generated Answer" :prediction_answer_list[i]['answer'],
                              "Predicted Grade" :graded_outputs[i]['text']}
                                       )

human_incorrect_ans_review_df = pd.DataFrame(human_incorrect_ans_review)
human_correct_ans_review_df = pd.DataFrame(human_correct_ans_review)

print("Machine Evaluation Total Score: " + str(int(total_correct_counter*100/len(graded_outputs))) + "%")
print("Machine Evaulation Total Incorrect Ans: " + str(int(total_incorrect_counter*100/len(graded_outputs))) + "%")

Evaluation Total Score: 72%
Evaulation Total Incorrect Ans: 27%


In [46]:
human_incorrect_ans_review_df

,Generate-Index,Generated Question,Context Answer,Generated Answer,Predicted Grade
0,6,What types of countries do you offer international shipping to?,"""Yes, we offer international shipping to select countries. The availability and shipping costs will be calculated during the checkout process based on your location.""""Yes, we offer international shipping to select countries. Please review the available shipping destinations during checkout or contact our customer support for assistance.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","INCORRECT\n\nThe chatbot answer does not provide any relevant information about international shipping countries and instead directs the customer to contact human service. The context clearly states that the company offers international shipping to select countries, but the chatbot's response creates unnecessary uncertainty and redirects the customer for assistance."
1,8,What steps should be taken if you need to change your shipping address after placing an order?,"""If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.""""If you need to change your order, please contact our customer support team as soon as possible. We will do our best to accommodate your request if the order has not been processed yet.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","CUSTOMER QUESTION: What steps should be taken if you need to change your shipping address after placing an order?\nCONTEXT: ""If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.""""If you need to change your order, please contact our customer support team as soon as possible. We will do our best to accommodate your request if the order has not been processed yet.""\nCUSTOMER SERVICE CHATBOT ANSWER: I am not sure about the answer, please contact our human service for assistance. Thank You!\nGRADE: INCORRECT"
2,11,What is the company's policy for matching prices found on competitor websites?,"""Our privacy policy outlines how we collect, use, and protect your personal information. Please visit our Privacy Policy page for detailed information.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","CUSTOMER QUESTION: What is the company's policy for matching prices found on competitor websites?\nCONTEXT: ""Our privacy policy outlines how we collect, use, and protect your personal information. Please visit our Privacy Policy page for detailed information.""\nCUSTOMER SERVICE CHATBOT ANSWER: I am not sure about the answer, please contact our human service for assistance. Thank You!\nGRADE: INCORRECT"
3,15,Does the company have a loyalty program?,"""To apply for a job at our company, visit our Careers page, where you can find current job openings and submit your application.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","CUSTOMER QUESTION: Does the company have a loyalty program?\nCONTEXT: ""To apply for a job at our company, visit our Careers page, where you can find current job openings and submit your application.""\nCUSTOMER SERVICE CHATBOT ANSWER: I am not sure about the answer, please contact our human service for assistance. Thank You!\nGRADE: INCORRECT"
4,17,Does the company offer bulk or wholesale discounts?,"""Yes, we offer bulk or wholesale discounts for certain products. Please contact our customer support team or visit our Wholesale page for more information and to discuss your specific requirements.""""Yes, we have a loyalty program where you can earn points for every purchase. These points can be redeemed for discounts on future orders. Please visit our website to learn more and join the program.""","Yes, we offer bulk or wholesale disc

### Manual Evaluation / Review

In [49]:
# langchain.debug = True
# memory.clear()
# qa.invoke(new_examples[2]['qa_pairs']["query"])

In [50]:
# langchain.debug = False

In [47]:
# Filter the human_incorrect_ans_review df to those the chatbot said he did not know the answer
# We will add those answers to the data to expand and improve the chatbot's RAG knowledge capacity

dunno_ans_df = human_incorrect_ans_review_df[human_incorrect_ans_review_df['Generated Answer'].str.contains("I am not sure about the answer", case=False)]

In [48]:
dunno_ans_df.reset_index(inplace=True)

,index,Generate-Index,Generated Question,Context Answer,Generated Answer,Predicted Grade
0,0,6,What types of countries do you offer international shipping to?,"""Yes, we offer international shipping to select countries. The availability and shipping costs will be calculated during the checkout process based on your location.""""Yes, we offer international shipping to select countries. Please review the available shipping destinations during checkout or contact our customer support for assistance.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","INCORRECT\n\nThe chatbot answer does not provide any relevant information about international shipping countries and instead directs the customer to contact human service. The context clearly states that the company offers international shipping to select countries, but the chatbot's response creates unnecessary uncertainty and redirects the customer for assistance."
1,1,8,What steps should be taken if you need to change your shipping address after placing an order?,"""If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.""""If you need to change your order, please contact our customer support team as soon as possible. We will do our best to accommodate your request if the order has not been processed yet.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","CUSTOMER QUESTION: What steps should be taken if you need to change your shipping address after placing an order?\nCONTEXT: ""If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.""""If you need to change your order, please contact our customer support team as soon as possible. We will do our best to accommodate your request if the order has not been processed yet.""\nCUSTOMER SERVICE CHATBOT ANSWER: I am not sure about the answer, please contact our human service for assistance. Thank You!\nGRADE: INCORRECT"
2,2,11,What is the company's policy for matching prices found on competitor websites?,"""Our privacy policy outlines how we collect, use, and protect your personal information. Please visit our Privacy Policy page for detailed information.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","CUSTOMER QUESTION: What is the company's policy for matching prices found on competitor websites?\nCONTEXT: ""Our privacy policy outlines how we collect, use, and protect your personal information. Please visit our Privacy Policy page for detailed information.""\nCUSTOMER SERVICE CHATBOT ANSWER: I am not sure about the answer, please contact our human service for assistance. Thank You!\nGRADE: INCORRECT"
3,3,15,Does the company have a loyalty program?,"""To apply for a job at our company, visit our Careers page, where you can find current job openings and submit your application.""","I am not sure about the answer, please contact our human service for assistance. Thank You!","CUSTOMER QUESTION: Does the company have a loyalty program?\nCONTEXT: ""To apply for a job at our company, visit our Careers page, where you can find current job openings and submit your application.""\nCUSTOMER SERVICE CHATBOT ANSWER: I am not sure about the answer, please contact our human service for assistance. Thank You!\nGRADE: INCORRECT"
4,5,18,Can I change or cancel an item in my order?,"""If you need to change or cancel an item in your order, please contact our customer support team as soon as possible. We will assist you with the necessary steps.""""You can cancel your order if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the cancellation process.""""If you need to change your order, please contact our customer support team as soon as

In [57]:
# Filter the human_incorrect_ans_review df to those the chatbot provided maybe truly incorrect answer
# We will review those manually. 
# If it is actually correct, we will move it to the correct list
# If it is actually incorrect, we will add the data to expand and improve the chatbot's RAG knowledge capacity

human_filtered_incorrect_ans_review_df = human_incorrect_ans_review_df[~human_incorrect_ans_review_df['Generate-Index'].isin(dunno_ans_df['Generate-Index'])]

In [59]:
human_filtered_incorrect_ans_review_df.reset_index()

,index,Generate-Index,Generated Question,Context Answer,Generated Answer,Predicted Grade
0,4,17,Does the company offer bulk or wholesale discounts?,"""Yes, we offer bulk or wholesale discounts for certain products. Please contact our customer support team or visit our Wholesale page for more information and to discuss your specific requirements.""""Yes, we have a loyalty program where you can earn points for every purchase. These points can be redeemed for discounts on future orders. Please visit our website to learn more and join the program.""","Yes, we offer bulk or wholesale discounts for certain products. Please contact our customer support team or visit our Wholesale page for more information and to discuss your specific requirements.","GRADE: INCORRECT\n\nThe Chatbot's answer provides all the required context about bulk or wholesale discounts in the question being asked but also includes additional details that are not relevant to the specific query made. Specifically, it mentions a loyalty program with points redeemable for future orders and encourages visitors to join the program on the website. These details exceed what is necessary to address the customer's direct inquiry."
1,8,27,Can a customer order a product as a gift and have it shipped directly to the intended recipient?,"""Yes, you can order a product as a gift and have it shipped directly to the recipient. During the checkout process, you can enter the recipient's shipping address.""""Yes, you can add a gift message during the checkout process. There is usually a section where you can enter your personalized message.""","Yes, you can order a product as a gift and have it shipped directly to the recipient. During the checkout process, you can enter the recipient's shipping address.","CUSTOMER QUESTION: Can a customer order a product as a gift and have it shipped directly to the intended recipient?\nCONTEXT: ""Yes, you can order a product as a gift and have it shipped directly to the recipient. During the checkout process, you can enter the recipient's shipping address.""\nCUSTOMER SERVICE CHATBOT ANSWER: Yes, you can order a product as a gift and have it shipped directly to the recipient. During the checkout process, you can enter the recipient's shipping address.\nGRADE: INCORRECT\n\nThe chatbot answer provides all the context information provided in the question, including the option to add a personalized message during the checkout process, which was not mentioned in the original context."
2,15,46,What should I do if I receive a damaged product?,"""If your product was damaged during shipping, please contact our customer support team immediately. We will guide you through the return and replacement process.""""If your product was damaged due to mishandling during shipping, please contact our customer support team immediately. We will assist you with the necessary steps for return and replacement.""","If you receive a damaged product, please contact our customer support team immediately. We will guide you through the return and replacement process.","CUSTOMER QUESTION: What should I do if I receive a damaged product?\nCONTEXT: ""If your product was damaged during shipping, please contact our customer support team immediately. We will guide you through the return and replacement process.""""If your product was damaged due to mishandling during shipping, please contact our customer support team immediately. We will assist you with the necessary steps for return and replacement.""\nCUSTOMER SERVICE CHATBOT ANSWER: If you receive a damaged product, please contact our customer support team immediately. We will guide you through the return and replacement process.\nGRADE: INCORRECT"
3,21,63,"What is typically the policy regarding returns on products that are marked as ""clearance"" or ""final sale""?","""Clearance or final sale items are typically non-returnable and non-refundable. Please review the product description or contact our customer support team for more 

### Human Correction

In [66]:
# Enter the Generate-Index of the questions you want to move into the correct list and those you want to keep in the incorrect list
human_correct_list_generate_index = [17, 27, 46, 63, 76, 85, 96, 117, 123, 127, 146, 173, 187]

# Create a df and change the Predicted Grade to CORRECT
human_correct_df = human_filtered_incorrect_ans_review_df[human_filtered_incorrect_ans_review_df['Generate-Index'].isin(human_correct_list_generate_index)]
human_correct_df["Predicted Grade"] = "HUMAN REVIEWED GRADE: CORRECT"

human_correct_df

,Generate-Index,Generated Question,Context Answer,Generated Answer,Predicted Grade
4,17,Does the company offer bulk or wholesale discounts?,"""Yes, we offer bulk or wholesale discounts for certain products. Please contact our customer support team or visit our Wholesale page for more information and to discuss your specific requirements.""""Yes, we have a loyalty program where you can earn points for every purchase. These points can be redeemed for discounts on future orders. Please visit our website to learn more and join the program.""","Yes, we offer bulk or wholesale discounts for certain products. Please contact our customer support team or visit our Wholesale page for more information and to discuss your specific requirements.",HUMAN REVIEWED GRADE: CORRECT
8,27,Can a customer order a product as a gift and have it shipped directly to the intended recipient?,"""Yes, you can order a product as a gift and have it shipped directly to the recipient. During the checkout process, you can enter the recipient's shipping address.""""Yes, you can add a gift message during the checkout process. There is usually a section where you can enter your personalized message.""","Yes, you can order a product as a gift and have it shipped directly to the recipient. During the checkout process, you can enter the recipient's shipping address.",HUMAN REVIEWED GRADE: CORRECT
15,46,What should I do if I receive a damaged product?,"""If your product was damaged during shipping, please contact our customer support team immediately. We will guide you through the return and replacement process.""""If your product was damaged due to mishandling during shipping, please contact our customer support team immediately. We will assist you with the necessary steps for return and replacement.""","If you receive a damaged product, please contact our customer support team immediately. We will guide you through the return and replacement process.",HUMAN REVIEWED GRADE: CORRECT
21,63,"What is typically the policy regarding returns on products that are marked as ""clearance"" or ""final sale""?","""Clearance or final sale items are typically non-returnable and non-refundable. Please review the product description or contact our customer support team for more information.""""Final sale items are usually non-returnable and non-refundable. Please review the product description or contact our customer support team to confirm the return eligibility for specific items.""",Final sale items are usually non-returnable and non-refundable. Please review the product description or contact our customer support team to confirm the return eligibility for specific items.,HUMAN REVIEWED GRADE: CORRECT
24,76,What should you do if a product you want is not available in your preferred color?,"""If a product is not available in your preferred color, it may be temporarily out of stock. Please check back later or sign up for color notifications.""""If a product is currently out of stock, you will usually see an option to sign up for product notifications. This way, you will be alerted when the product becomes available again.""","If a product you want is not available in your preferred color, please check back later or sign up for color notifications.",HUMAN REVIEWED GRADE: CORRECT
28,85,What type of guarantee does the company offer for their products?,"""Yes, we offer a satisfaction guarantee on our products. If you are not satisfied with your purchase, please contact our customer support team for assistance.""",We offer a satisfaction guarantee on our products.,HUMAN REVIEWED GRADE: CORRECT
30,96,What are the steps to apply for a job at this company?,"""To apply for a job at our company, visit our Careers page, where you can find current job openings and submit your application.""","To apply for a job at ABC-Company, please visit our Careers page. You can find current job openings and submit your application there.",HUMAN REVIEWED GRADE: CORRECT
34,117,What is the general policy regard

In [67]:
human_correct_ans_final_df = pd.concat([human_correct_ans_review_df, human_correct_df])

In [68]:
human_correct_ans_final_df

,Generate-Index,Generated Question,Context Answer,Generated Answer,Predicted Grade
0,0,What is the step-by-step process for creating an account on this website?,"""To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.""""Yes, you can place an order as a guest without creating an account. However, creating an account offers benefits such as order tracking and easier future purchases.""","To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.","CUSTOMER QUESTION: What is the step-by-step process for creating an account on this website?\nCONTEXT: ""To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.""""Yes, you can place an order as a guest without creating an account. However, creating an account offers benefits such as order tracking and easier future purchases.""\nCUSTOMER SERVICE CHATBOT ANSWER: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.\nGRADE: CORRECT"
1,1,What payment methods does an online retailer typically accept for processing online orders?,"""We accept major credit cards, debit cards, and PayPal as payment methods for online orders.""""Yes, we take the security of your personal and payment details seriously. We use industry-standard encryption and follow strict security protocols to ensure your information is protected.""","We accept major credit cards, debit cards, and PayPal as payment methods for online orders.","CUSTOMER QUESTION: What payment methods does an online retailer typically accept for processing online orders?\nCONTEXT: ""We accept major credit cards, debit cards, and PayPal as payment methods for online orders.""""Yes, we take the security of your personal and payment details seriously. We use industry-standard encryption and follow strict security protocols to ensure your information is protected.""\nCUSTOMER SERVICE CHATBOT ANSWER: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.\nGRADE: CORRECT"
2,2,What steps should I take to track my order?,"""You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.""""If a product is available for pre-order, you can place an order to secure your item. The product will be shipped once it becomes available.""","To track your order, please log into your account and navigate to the 'Order History' section. There, you will find the tracking information for your shipment.","CUSTOMER QUESTION: What steps should I take to track my order?\nCONTEXT: ""You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment.""""If a product is available for pre-order, you can place an order to secure your item. The product will be shipped once it becomes available.""\nCUSTOMER SERVICE CHATBOT ANSWER: To track your order, please log into your account and navigate to the 'Order History' section. There, you will find the tracking information for your shipment.\nGRADE: CORRECT"
3,3,What is the timeframe within which customers can initiate a return under the company's return policy?,"""Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.""""Yes, you can return a product purchased during a promotional event. The refund will be processed based on the amount paid after any applicable discounts.""","Customers can initiate a return within 30 days of purchase for a full

In [69]:
len(human_correct_df)

13

In [70]:
# Calculate Chatbot performance score again after human review

reviewed_correct_counter = total_correct_counter + len(human_correct_df)
reviewed_incorrect_counter = total_incorrect_counter - len(human_correct_df)

print("Human Reviewed Evaluation Total Score: " + str(int(reviewed_correct_counter*100/len(graded_outputs))) + "%")
print("Human Reviewed Evaulation Total Incorrect Ans: " + str(int(reviewed_incorrect_counter*100/len(graded_outputs))) + "%")

Human Reviewed Evaluation Total Score: 79%
Human Reviewed Evaulation Total Incorrect Ans: 21%


### Add generated qa pairs into dataset
- After human reviewing the evaluated incorrect answers, to improve the llm chatbot, we will add the selected generated qa pairs into the dataset for the next round of evaluation

In [84]:
print(generated_qa_examples_df["Dataset Index"].dtype)
print(human_correct_ans_final_df["Generate-Index"].dtype)

object
int64


In [85]:
generated_qa_examples_df["Dataset Index"] = generated_qa_examples_df["Dataset Index"].astype(int)
human_correct_ans_final_df["Generate-Index"] = human_correct_ans_final_df["Generate-Index"].astype(int)

data_to_add_df = generated_qa_examples_df[~generated_qa_examples_df["Dataset Index"].isin(human_correct_ans_final_df["Generate-Index"])]

In [111]:
data_to_add_df
data_to_add_df_no_duplicate = data_to_add_df.drop_duplicates()
data_to_add_df_no_duplicate.reset_index(drop=True, inplace=True)

In [184]:
data_deep_copy = copy.deepcopy(data)
# data_deep_copy

In [185]:
for i in range(len(data_to_add_df_no_duplicate)):
    data_to_append = Document(metadata={'question': data_to_add_df_no_duplicate["Generated Question"][i]},
                       # page_content=data_to_add_df_no_duplicate["Generated Answer"][i]
                      page_content = "Example Question: " + data_to_add_df_no_duplicate["Generated Question"][i] + " " + "Example Answer: " + data_to_add_df_no_duplicate["Generated Answer"][i]
                             )
    data_deep_copy.append(data_to_append)


In [187]:
print(len(data))
print(len(data_to_add_df_no_duplicate))
print(len(data_deep_copy))

200
42
242


### Save the new dataset as a jsonl file for next round's evaluation use

In [188]:
def save_docs_to_jsonl(array:Iterable[Document], file_path:str)->None:
    with open(file_path, 'w') as jsonl_file:
        for doc in array:
            jsonl_file.write(doc.json() + '\n')

def load_docs_from_jsonl(file_path)->Iterable[Document]:
    array = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            obj = Document(**data)
            array.append(obj)
    return array

In [189]:
save_docs_to_jsonl(data_deep_copy,'dataset_with_appended_new_qa.jsonl')

In [203]:
# data_to_add_df

In [193]:
# Save as CSV
# data_to_add_df.to_csv("data_to_add_df.csv", index=False)

### Test loading the new dataset to make sure it works

In [190]:
dataset_with_appended_new_qa=load_docs_from_jsonl('dataset_with_appended_new_qa.jsonl')
print(len(dataset_with_appended_new_qa))

242


In [206]:
dataset_with_appended_new_qa[-10:]

[Document(metadata={'question': 'Can you still make changes to an order that has already been processed?'}, page_content='Example Question: Can you still make changes to an order that has already been processed? Example Answer: No, according to our policy, you should contact customer support as soon as possible if you need to change your order and it has not yet been processed.'),
 Document(metadata={'question': 'What information can be found on the teacher\'s "Privacy Policy" page?'}, page_content='Example Question: What information can be found on the teacher\'s "Privacy Policy" page? Example Answer: The page provides detailed information on how the teacher collects, uses, and protects personal information.'),
 Document(metadata={'question': 'What is the warranty policy regarding your products?'}, page_content='Example Question: What is the warranty policy regarding your products? Example Answer: The warranty policy varies by item, and customers should consult the product page or con